# Um exemplo

- Suponha que queremos decidir entre 3 tarefas, fazer ou não fazer, e obter um valor máximo.

$x, y, z$ são tarefas

- Você precisa escolher a tarefa 1 ou a tarefa 2, ou ambas.

$
x + y \geq 1
$

- O tempo total que você tem para realizar essas tarefas são 4 horas.

    - A tarefa 1 toma 1 hora.

    - A tarefa 2 toma 2 horas.

    - A tarefa 3 toma 3 horas.

$
1 x + 2 y + 3 z \leq 4
$

- A tarefa 3 vale o dobro da tarefa 1 e da tarefa 2.
 
$
\max \ x + y + 2 z
$

- Este problema pode ser modelado como um problema de programação inteira binária.

$
x, y, z \in \{ 0, 1 \}
$

- Formulação

$
\begin{align}
\max \ & x + y + 2 z \\
& x + 2 y + 3 z \leq 4 \\
& x + y \geq 1 \\
& x, y, z \in \{ 0, 1 \}
\end{align}
$

# Modelando e resolvendo o problema

- Defina o modelo.

- Defina as variáveis de decisão.

- Defina a função objetivo.

- Adicionando as restrições.

- Execute a função optimize().

- Imprima as soluções.

In [2]:
# importando bibliotecas
import gurobipy as gp
from gurobipy import GRB

In [3]:
# definindo o modelo
mip0 = gp.Model("mip0")

Academic license - for non-commercial use only - expires 2022-09-29
Using license file /opt/gurobi912/gurobi.lic


In [4]:
# definindo variáveis
x = mip0.addVar(vtype=GRB.BINARY, name="x")
y = mip0.addVar(vtype=GRB.BINARY, name="y")
z = mip0.addVar(vtype=GRB.BINARY, name="z")

In [5]:
# definindo a função objetivo
mip0.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

In [6]:
# adicionando a restrição, x + 2 y + 3 z <= 4
mip0.addConstr(x + 2 * y + 3 * z <= 4, "rest0")

# adicionando a restrição, x + y >= 1
mip0.addConstr(x + y >= 1, "rest1")

<gurobi.Constr *Awaiting Model Update*>

In [7]:
# exportando .lp/.mps
mip0.write("mip0.lp")

In [8]:
# resolvendo o modelo
mip0.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.31s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.50 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [9]:
# imprimindo a solução ótima
print('Obj = %g' % mip0.objVal)

# imprimindo as soluções
for v in mip0.getVars():
    print('%s = %g' % (v.varName, v.x))

Obj = 3
x = 1
y = 0
z = 1


# Lendo/Resolvendo .mps ou .lp

In [10]:
# Criando o modelo de um arquivo(mps)
model = gp.read("mps/afiro.mps")

Read MPS format model from file mps/afiro.mps
Reading time = 0.01 seconds
AFIRO: 27 rows, 32 columns, 83 nonzeros


In [11]:
# resolvendo o modelo
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 27 rows, 32 columns and 83 nonzeros
Model fingerprint: 0x0e972b37
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [3e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 5e+02]
Presolve removed 18 rows and 20 columns
Presolve time: 0.03s
Presolved: 9 rows, 12 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.9751945e+02   5.541356e+01   0.000000e+00      0s
       6   -4.6475314e+02   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.06 seconds
Optimal objective -4.647531429e+02


In [12]:
# imprimindo o valor ótimo
model.Objval

-464.75314285714285

In [13]:
# imprimindo a solução
model.printAttr('x')


    Variable            x 
-------------------------
         X01           80 
         X02         25.5 
         X03         54.5 
         X04         84.8 
         X06      18.2143 
         X14      18.2143 
         X16      19.3071 
         X22          500 
         X23       475.92 
         X24        24.08 
         X26          215 
         X36      339.943 
         X37      383.943 


# Modelo usando a idéia de matriz.

In [16]:
!pip3 install scipy

     |████████████████████████████████| 28.4 MB 75 kB/s  eta 0:00:01


In [17]:
import numpy as np
import scipy.sparse as sp

# criando um novo modelo
m1 = gp.Model("matrix")

# criando as variáveis
x = m1.addMVar(shape=3, vtype=GRB.BINARY, name="x")

# definindo a função objetivo
obj = np.array([1.0, 1.0, 2.0])
m1.setObjective(obj @ x, GRB.MAXIMIZE)

# construindo a matriz de restrição, esparsa
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

# construindo o vetor do lado direito, rhs
rhs = np.array([4.0, -1.0])

# adicionando as restrições
m1.addConstr(A @ x <= rhs, name="c")
    
# gerando .lp
m1.write("matrix.lp")

# resolvendo o problema
m1.optimize()

# imprimindo a solução ótima
print(x.X)
    
# imprimindo o valor ótimo
print('Obj = %g' % m1.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x8d4960d3
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
[1. 0. 1.]
Obj = 3
